In [1]:
import h5py
import pyccl
import scipy
import numpy
import json

In [2]:
# Load in Data

PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'
DATA_PATH = PATH + 'DATA/'
CODE_PATH = PATH + 'CODE/'
PLOT_PATH = PATH + 'PLOT/'

BIN_SIZE = 4
GRID_SIZE = 72
LATENT_SIZE = 32
DATA_SIZE = 100000
COLOR = ['blue', 'green', 'orange', 'red']

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as FILE:
    
    Z_GRID = numpy.array(FILE['realizations/z'][:GRID_SIZE + 1], dtype = 'double')
    GRID = numpy.array(FILE['realizations/pdfs'][:, :BIN_SIZE, :GRID_SIZE + 1], dtype = 'double')

In [3]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as FILE:
    COSMO = json.load(FILE)
    
COSMO_CCL = pyccl.cosmology.Cosmology(
    h=COSMO['H'],
    w0=COSMO['W0'],
    wa=COSMO['WA'],
    n_s=COSMO['NS'],
    m_nu=COSMO['MNU'],
    Neff=COSMO['NEFF'],
    T_CMB=COSMO['TCMB'],
    mass_split='single',
    sigma8=COSMO['SIGMA8'],
    Omega_k=COSMO['OMEGAK'],
    Omega_c=COSMO['OMEGAC'],
    Omega_b=COSMO['OMEGAB'],
    mg_parametrization=None,
    matter_power_spectrum='halofit',
    transfer_function='boltzmann_camb',
    extra_parameters={
        'camb': {'kmax': 10000, 'lmax': 10000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# PMM, matter-matter-power-spectra

PSI_GRID = numpy.mean(GRID, axis = 0)
SIGMA_GRID = numpy.std(GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + Z_GRID))
CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + Z_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 100
ELL_DATA = numpy.logspace(1, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where = CHI_MESH > 0))

PMM_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):

    PMM_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [5]:
#numerical integral

def integralI5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    def formulan(chi):
        
        result = ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))**2

        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result

    def formula0(chi):
        
        result = ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))**2

        result = result * (((chi/chi2)**3)*power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result
   
    if chi1 == 0:
        
        integral, error = scipy.integrate.quad_vec(f = formula0, a = chi1, b = chi2)
        
    else:
        
        integral, error = scipy.integrate.quad_vec(f = formulan, a = chi1, b = chi2)

    return integral

In [6]:
#coefficient equation

def coefficientJ5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    y = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)

    if a == 1:

        formula = (1 / (10080 * b**2)) * (b**2 * (1785 + z * ( - 562 + 79 * z) + 42 * b**2 * (15 + ( - 6 + z) * z) + 6 * b * (350 + z * ( - 124 + 19 * z))) - 
8 * b * (1 + b) * (432 + z * ( - 129 + 17 * z) + 12 * b * (21 + ( - 7 + z) * z)) * numpy.log(1 + b) + 60 * (1 + b)**2 * (28 + ( - 8 + z) * z) * numpy.log(1 + b)**2)

    else:

        formula =  (1 / (5292000 * a**5 * b**2)) * (a * b**2 * (245 * a**2 * (420 * y + a * ( - 600 + a**4 * (4030 - 2439 * y) - 90 * (39 + 40 * b) * y + 5 * a**2 * ( - 220 - 60 * b * ( - 2 * ( - 9 + y) + 9 * b * ( - 2 + y)) + 133 * y) + 
a**3 * ( - 2150 + 300 * b * (30 - 17 * y) + 404 * y) + 20 * a * (495 + 34 * y + 90 * b * (6 + y)))) + 
14 * a * ( - 10920 * y - 70 * a**3 * ( - 340 + 150 * b * ( - 6 + y) + 163 * y) + a**6 * ( - 85365 + 60096 * y) + 420 * a * (35 + 3 * (53 + 50 * b) * y) - 
70 * a**2 * (1755 + 232 * y + 450 * b * (4 + y)) + 14 * a**5 * (1010 - 303 * y + 75 * b * ( - 170 + 117 * y)) + 
7 * a**4 * (3325 - 952 * y + 750 * b * (4 - y + 6 * b * ( - 3 + 2 * y)))) * z + 
(a**7 * (420672 - 322375 * y) + 59220 * y + 35 * a**3 * ( - 3248 + 1260 * b * ( - 5 + y) + 1371 * y) + a**6 * ( - 29694 + 4410 * b * (195 - 148 * y) + 11624 * y) - 
210 * a * (364 + 3 * (481 + 420 * b) * y) + 14 * a**4 * ( - 5705 + 1941 * y + 525 * b * ( - 10 + 3 * y)) + 420 * a**2 * (1113 + 197 * y + 105 * b * (10 + 3 * y)) - 
7 * a**5 * (6664 - 2441 * y + 210 * b * (25 - 9 * y + 75 * b * ( - 4 + 3 * y)))) * z**2) + 
420 * ( - 210 * ( - 1 + a)**5 * b**2 * (5 * a**2 * (y + a * ( - 4 + 3 * y)) - 2 * a * (2 * y + a * ( - 5 + 6 * y + 3 * a * ( - 5 + 4 * y))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a**2) + y + a * (3 + 2 * a * (3 + 5 * a)) * y) * z**2) * numpy.log(1 - a)**2 + ( - 1 + a)**3 * b * numpy.log(1 - a) * (b * (35 * a**2 * ( - 10 * a * ( - 1 + a * (2 + 17 * a + 18 * b)) + ( - 7 + a * (11 + 60 * b + a * (59 + 117 * a + 120 * b))) * y) - 
14 * a * ( - 5 * a * ( - 7 + a * (11 + 60 * b + a * (59 + 117 * a + 120 * b))) + 2 * ( - 13 + a * (17 + 75 * b + a * (77 + 150 * b + 3 * a * (49 + 74 * a + 75 * b)))) * y) * z + 
( - 14 * a * ( - 13 + a * (17 + 75 * b + a * (77 + 150 * b + 3 * a * (49 + 74 * a + 75 * b)))) + 
( - 141 + a * (159 + 669 * a + 1249 * a**2 + 1864 * a**3 + 2500 * a**4 + 630 * (1 + a * (2 + a * (3 + 4 * a))) * b)) * y) * z**2) + 
420 * ( - 1 + a) * a * (1 + b) * (5 * a**2 * (y + a * ( - 4 + 3 * y)) - 2 * a * (2 * y + a * ( - 5 + 6 * y + 3 * a * ( - 5 + 4 * y))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a**2) + y + a * (3 + 2 * a * (3 + 5 * a)) * y) * z**2) * numpy.log(1 + b)) - 
a**2 * (1 + b) * numpy.log(1 + b) * (b * ( - 420 * y * z**2 + 210 * a * z * (4 * z + y * (8 + z)) + a**6 * ( - 5950 + 4095 * y + 42 * (195 - 74 * z) * z + 4 * y * z * ( - 1554 + 625 * z)) + 
70 * a**2 * ( - 6 * z * (10 + z) + y * ( - 30 + ( - 12 + z) * z)) + 35 * a**3 * (240 - 4 * ( - 15 + z) * z + y * (30 + ( - 8 + z) * z)) + 
7 * a**5 * ( - 25 * ( - 64 + 35 * y + 70 * z) + 2 * z * (y * (594 - 224 * z) + 297 * z) + 30 * b * ( - 30 + 20 * y + 5 * (8 - 3 * z) * z + 6 * y * z * ( - 5 + 2 * z))) + 
7 * a**4 * (50 * ( - 12 + y) - 20 * ( - 5 + y) * z + ( - 10 + 3 * y) * z**2 - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + y * (6 + z * ( - 8 + 3 * z))))) + 
210 * a**4 * (1 + b) * ( - 20 * (3 + ( - 3 + a) * a) + 5 * (6 + a * ( - 8 + 3 * a)) * y + 2 * (30 + 5 * a * ( - 8 + 3 * a) - 20 * y + 6 * (5 - 2 * a) * a * y) * z + 
(5 * ( - 4 + 3 * y) + 2 * a * (15 - 12 * y + a * ( - 6 + 5 * y))) * z**2) * numpy.log(1 + b))))
        

    coefficient = chi2**3 * power2 * (1 + redshift2)**2 * formula
    
    return coefficient

In [7]:
#Case1 test: For chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.30684999374 1497.8183354014277 1534.1133113296617 49919.45246058225 48958.62248873209 0.36 0.37


In [8]:
#Case1 test: For chi1 > 0

INTEGRAL = integralI5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(922716.5509601753, 922716.5608644981, 1.0733873567758678e-08)

In [9]:
#Case2 test: For chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.42674493673601 88.64121385461215 0.0 2114.2567965927065 0.0 0.01


In [10]:
#Case2 test: For chi1 = 0

INTEGRAL = integralI5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(2911468.129861286, 2911468.1298612477, -1.3100631690576847e-14)